In [11]:
# Instalar pyodbc
# import sys
# !{sys.executable} -m pip install pyodbc
# import sys
# !{sys.executable} -m pip install openpyxl
# !{sys.executable} -m pip install --upgrade seaborn
# !{sys.executable} -m pip install --upgrade matplotlib

In [12]:
#Librería pyodbc
import pyodbc
import pandas as pd
import numpy as np
import openpyxl

# Conexión al Access
# Creo función 
def py_conn():
    conn_str = (
        r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
        r'DBQ=C:\\Users\\MEC\\OneDrive - BCRA\\General\\GH\\clustering.accdb;'
        )
    conn = pyodbc.connect(conn_str)

    return conn


def py_conn_tables(cursor):
    for table_info in cursor.tables(tableType='TABLE'):
        print(table_info.table_name)

In [13]:
# Consulta tabla indicadores
conn = py_conn()
data = pd.read_sql_query("SELECT * FROM DATA", conn)
conn.close()
# Veo la tabla que me trajo
print(data)

C:\Users\MEC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


        EENTIDAD CODIND     FECHAS        IND
0             00     A1 2003-12-01  85.774463
1             00     A1 2004-01-01  84.701127
2             00     A1 2004-02-01  86.342852
3             00     A1 2004-03-01  88.700621
4             00     A1 2004-04-01  90.881437
...          ...    ...        ...        ...
1433247        8    RG9 2021-07-01  -7.141944
1433248        8    RG9 2021-08-01  -7.100442
1433249        8    RG9 2021-09-01  -7.357433
1433250        8    RG9 2021-10-01  -7.415377
1433251        8    RG9 2021-11-01  -7.303206

[1433252 rows x 4 columns]


In [14]:
# Elimino sistema, ghs y gh8
gh = ['1', '2', '3', '4', '5', '6', '7', '8', '11111', '00', '10', '11',
		'44099', '44098', '44096', '44095', '44088', '44093', '44092', '44090', '00340', '44059', '44094'
		]
data= data[~data['EENTIDAD'].isin(gh)]

In [15]:
# Elimino indicadores que no necesito
ind_cod = [
          'RG2', 'RG1', 'E2', 'E1', 'RAPM',
          'A6', 'AG1', 'A2', 'A21',
          'L1', 'L8',
          'C1', 'C14', 'C2', 
          'LS1', 'LS2', 'MS1'
		]
data = data[data['CODIND'].isin(ind_cod)]
print(data[data['CODIND']=='A6'])
print(data[data['CODIND']=='C14'])

Empty DataFrame
Columns: [EENTIDAD, CODIND, FECHAS, IND]
Index: []
        EENTIDAD CODIND     FECHAS          IND
16252      00003    C14 2004-01-01    77.653089
16253      00003    C14 2004-02-01    77.170798
16254      00003    C14 2004-03-01    77.170798
19001      00005    C14 2004-01-01  1259.475195
19002      00005    C14 2004-02-01   780.396639
...          ...    ...        ...          ...
1383669    65203    C14 2021-07-01   380.725291
1383670    65203    C14 2021-08-01   510.117471
1383671    65203    C14 2021-09-01   626.829243
1383672    65203    C14 2021-10-01   510.731854
1383673    65203    C14 2021-11-01   537.283682

[15110 rows x 4 columns]


In [16]:
# Datos desde 2013
data = data[data['FECHAS'].dt.strftime('%Y') >= '2013']
print(data)

        EENTIDAD CODIND     FECHAS        IND
17448      00005     A2 2013-01-01  20.070588
17449      00005     A2 2013-02-01  20.675029
17450      00005     A2 2013-03-01  21.914390
17451      00005     A2 2013-04-01  30.937750
17452      00005     A2 2013-05-01  33.333333
...          ...    ...        ...        ...
1392519    65203    RG2 2021-07-01  62.946652
1392520    65203    RG2 2021-08-01  66.162508
1392521    65203    RG2 2021-09-01  66.941748
1392522    65203    RG2 2021-10-01  67.309958
1392523    65203    RG2 2021-11-01  68.432335

[100381 rows x 4 columns]


In [17]:
# Elimino entidades NO vigentes
conn = py_conn()
entidad = pd.read_sql_query("SELECT * FROM ENTIDAD", conn)
conn.close()
vigente = entidad['EENTIDAD'][entidad['VIGENTE']==True]
# print(vigente)
data = data[data['EENTIDAD'].isin(vigente)]

C:\Users\MEC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [18]:
# Cuantos valores nulos hay
print(f'Cantidad de valores nulos por columna:\n{data.isna().sum()}') # Chequear RAPM

Cantidad de valores nulos por columna:
EENTIDAD    0
CODIND      0
FECHAS      0
IND         0
dtype: int64


In [19]:
# Conjunto de indicadores con valore no nulos
data = data[data.isna()==False]

# Creo la columna de año, saco promedio de cada diciembre
data['%Y'] = data['FECHAS'].dt.strftime('%Y')
data_dic_mean = data
data_dic_mean['IND'].replace(0, np.nan)
data_dic_mean = data_dic_mean.groupby(['EENTIDAD', 'CODIND', '%Y']).mean()

# Tabla con promedios totales
data_mean = data.replace(0, np.nan)
data_mean = data.drop(columns=['FECHAS']).groupby(['EENTIDAD', 'CODIND'], as_index=False).mean()
# data_mean = data_mean.replace(0, np.nan)
# print(data_mean)

print(f'Cantidad de valores nulos por columna:\n{data.isna().sum()}')
print(f'Cantidad de valores nulos por columna:\n{data_dic_mean.isna().sum()}')
print(f'Cantidad de valores nulos por columna:\n{data_mean.isna().sum()}')

data_mean = data_mean.reset_index()
data_dic_mean = data_dic_mean.reset_index()

Cantidad de valores nulos por columna:
EENTIDAD    0
CODIND      0
FECHAS      0
IND         0
%Y          0
dtype: int64
Cantidad de valores nulos por columna:
IND    0
dtype: int64
Cantidad de valores nulos por columna:
EENTIDAD    0
CODIND      0
IND         0
dtype: int64


In [21]:
# Guardo excel con tablas de datos individuales y promedios
with pd.ExcelWriter('output/data.xlsx') as writer:  
    data.to_excel(writer, sheet_name='data')
    data_dic_mean.to_excel(writer, sheet_name='data_dic_mean')
    data_mean.to_excel(writer, sheet_name='data_mean')

In [22]:
# Creo datos para histogramas en Excel

ind = list(set(data_dic_mean['CODIND']))
periodo = sorted(list(set(data_dic_mean['%Y'])))

ind_per = [(i, p) for i in ind for p in periodo]

with pd.ExcelWriter('output/data.xlsx') as writer: 
    for ind, per in ind_per:
        if per == min(periodo):
            data_xlsx = data_dic_mean[data_dic_mean['CODIND']==ind][data_dic_mean['%Y']==per].reset_index()

        else:
            data_xlsx = pd.concat([data_xlsx, 
            data_dic_mean[data_dic_mean['CODIND']==ind][data_dic_mean['%Y']==per].reset_index()], axis=1)

        if per == max(periodo):
            data_xlsx.to_excel(writer, sheet_name=str(ind), index=False)

C:\Users\MEC\AppData\Local\Temp\ipykernel_12644\2440701672.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_xlsx = data_dic_mean[data_dic_mean['CODIND']==ind][data_dic_mean['%Y']==per].reset_index()
C:\Users\MEC\AppData\Local\Temp\ipykernel_12644\2440701672.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_dic_mean[data_dic_mean['CODIND']==ind][data_dic_mean['%Y']==per].reset_index()], axis=1)
C:\Users\MEC\AppData\Local\Temp\ipykernel_12644\2440701672.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_dic_mean[data_dic_mean['CODIND']==ind][data_dic_mean['%Y']==per].reset_index()], axis=1)
C:\Users\MEC\AppData\Local\Temp\ipykernel_12644\2440701672.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_dic_mean[data_dic_mean['CODIND']==ind][data_dic_mean['%Y']==per].reset_index()], axis=1)
C:\Users\MEC\AppData\Local\Temp\ipykernel_1264

In [24]:
# Creo tabla pivot serie de tiempo y grafico
from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

indicador = list(set(data_dic_mean['CODIND']))
periodo = sorted(list(set(data_dic_mean['%Y'])))
ind_per = [(i, p) for i in indicador for p in periodo]
ind = 'E1'
plt.style.use('ggplot')
plt.tight_layout()
plt.suptitle(f'Distribución {ind} promedio diciembre desde {min(periodo)} hasta {max(periodo)}')
# for ind in indicador: 
data_dic_mean_pivot = data_dic_mean[data_dic_mean['CODIND']==ind].pivot(index='EENTIDAD', columns='%Y', values='IND').reset_index()
data_dic_mean_pivot = data_dic_mean_pivot.set_index('EENTIDAD')
data_dic_mean_pivot = data_dic_mean_pivot.fillna(0.0)
sns.pairplot(data=data_dic_mean_pivot)
plt.savefig(f'graph/hist-scatter-{ind}-{min(periodo)}-{max(periodo)}.jpg')
plt.clf() # Clean parirplot figure from sns 
Image(filename=f'graph/hist-scatter-{ind}-{min(periodo)}-{max(periodo)}.jpg') # Show pairplot as image

In [ ]:
# Creo tabla pivot serie de tiempo
data_mean_pivot = data_mean.pivot(index='EENTIDAD', columns='CODIND', values='IND').reset_index()
data_mean_pivot = data_mean_pivot.set_index('EENTIDAD')
print(data_mean_pivot.isna().sum())

'''plt.style.use('ggplot')
plt.tight_layout()
plt.suptitle(f'Distribución {ind} promedio diciembre desde {min(periodo)} hasta {max(periodo)}')
sns.pairplot(data=data_mean_pivot)
plt.savefig(f'graph/hist-scatter-data-mean-{min(periodo)}-{max(periodo)}.jpg')'''

In [ ]:
'''# Busco valores nulos y reemplazo con cero
print(f'Cantidad de valores nulos por columna:\n{data_mean.isna().sum()}')
data_mean = data_mean.fillna(0)
print(f'Cantidad de valores nulos por columna:\n{data_mean.isna().sum()}')'''

In [ ]:
'''data_mean_pivot_describe = data_mean_pivot.describe()
data_mean_pivot_describe = data_mean_pivot_describe.transpose()
data_mean_pivot_describe = data_mean_pivot_describe.reset_index()'''